This Notebook serves the purpose of initial adhoc exploration of the data as well as generating the training data for later steps.

In [160]:
import os
import re

import pandas as pd
import numpy as np
import nltk
from copy import deepcopy

from nltk.tokenize import word_tokenize

import spacy

nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/barryz/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [161]:
corpus_directory = '/mnt/c/Course Data/Business Insider/2013-2014'
label_directory = '/mnt/c/Course Data/Business Insider/labels/'
labels = ['ceo.csv', 'companies.csv', 'percentage.csv']

In [162]:
ceos = ceos.fillna(0)
ceos['name'] = ceos.apply(lambda row: str(row[0]) + ' '+ str(row[1]) if row[1] != 0 else row[0], axis=1)
ceos_list = ceos.name.tolist()
company_list = companies[companies[0].notnull()][0].tolist()

In [163]:
corpus = ""
files = os.listdir(corpus_directory)
for file in files:
    with open(corpus_directory+'/' + file , 'r', encoding='latin1') as f:
        current = f.read()
        corpus += current

### CEO & Company 
- First identify all the people and organizations using SpaCy
- Then out of these, find the ones for CEO
- Tag the CEOs and these become the training samples for the new SpaCy
- In next Notebook, we will retrain Spacy using the found CEOs/Companies Tag


In [164]:
sentences = nltk.sent_tokenize(corpus)    

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/barryz/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-164-99f93e83a597>", line 1, in <module>
    sentences = nltk.sent_tokenize(corpus)
  File "/home/barryz/anaconda3/lib/python3.8/site-packages/nltk/tokenize/__init__.py", line 107, in sent_tokenize
    return tokenizer.tokenize(text)
  File "/home/barryz/anaconda3/lib/python3.8/site-packages/nltk/tokenize/punkt.py", line 1272, in tokenize
    return list(self.sentences_from_text(text, realign_boundaries))
  File "/home/barryz/anaconda3/lib/python3.8/site-packages/nltk/tokenize/punkt.py", line 1326, in sentences_from_text
    return [text[s:e] for s, e in self.span_tokenize(text, realign_boundaries)]
  File "/home/barryz/anaconda3/lib/python3.8/site-packages/nltk/tokenize/punkt.py", line 1326, in <listcomp>
    return [text[s:e] for s, e in self.span_tokenize(tex

TypeError: object of type 'NoneType' has no len()

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [175]:
# initial NER for each of the sentences

training = []

counter = 0
for sentence in sentences:
    counter += 1
    if counter % 50000 == 0:
        print(counter)
    doc = nlp(sentence)
    if len(doc) > 0:
        person_ent = []
        org_ent = [] 
        for ent in doc.ents:
            if ent.label_ == "ORG":
                if ent.text in company_list:
                    org_ent.append((ent.start_char, ent.end_char, "Company"))

                
            if ent.label_ == "PERSON":
                if ent.text in ceos_list:
                    person_ent.append((ent.start_char, ent.end_char, "CEO"))
        if len(person_ent+org_ent) != 0:    
            training.append((sentence, {"entities": person_ent+org_ent}))
                       

50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000


In [178]:
import csv
with open('/mnt/c/Course Data/Business Insider/total_training.csv', "w") as f:
    writer = csv.writer(f)
    writer.writerows(training)
f.close()
